a

In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient


from msrest.authentication import CognitiveServicesCredentials
import os
import json
import cv2

In [3]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''


def load_config(config_path='../config.json'):
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config


config = load_config()
subscription_key = config['subscription_key']
endpoint = config['endpoint']
#
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [4]:
folder_path = "images"  

def detect_bicycles(image_path, confidence_threshold=0.3):
    with open(image_path, "rb") as image_stream:
        result = computervision_client.detect_objects_in_stream(image_stream)
    
    bicycles = []
    print(f"Obiecte detectate în {image_path}:")
    for obj in result.objects:
        confidence = obj.confidence if hasattr(obj, 'confidence') else 1.0
        print(f" - {obj.object_property}: {confidence:.2f}")
        if obj.object_property.lower() == "bicycle" and confidence >= confidence_threshold:
            bicycles.append(obj)
    
    return bicycles

def draw_bounding_boxes(image_path, detected_objects):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    for obj in detected_objects:
        rect = obj.rectangle
        left = rect.x
        top = rect.y
        right = left + rect.w
        bottom = top + rect.h
        
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"bicycle: {obj.confidence:.2f}" if hasattr(obj, 'confidence') else "bicycle"
        cv2.putText(image, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image

files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg')) and 'bike' in f.lower()]


for file in files:
    image_path = os.path.join(folder_path, file)
    print(f"\nProcesez imaginea: {file}")
    
    detected_bicycles = detect_bicycles(image_path, confidence_threshold=0.5)
    
    if detected_bicycles:
        print(f"Am detectat {len(detected_bicycles)} biciclete:")
        for bike in detected_bicycles:
            confidence = bike.confidence if hasattr(bike, 'confidence') else 1.0
            print(f" - Bicicleta cu probabilitate: {confidence:.2f}")
        
        result_image = draw_bounding_boxes(image_path, detected_bicycles)
        
       
        output_path = os.path.join(folder_path, f"output_{file}")
        cv2.imwrite(output_path, result_image)
        print(f"Imagine salvata: {output_path}")
    else:
        print(f"Nu s-au detectat biciclete în {file}")


Procesez imaginea: bike02.jpg
Obiecte detectate în images\bike02.jpg:
 - Wheel: 0.52
 - bicycle: 0.91
Am detectat 1 biciclete:
 - Bicicleta cu probabilitate: 0.91
Imagine salvata: images\output_bike02.jpg

Procesez imaginea: bike03.jpg
Obiecte detectate în images\bike03.jpg:
 - car: 0.58
 - person: 0.79
 - person: 0.88
 - bicycle: 0.80
Am detectat 1 biciclete:
 - Bicicleta cu probabilitate: 0.80
Imagine salvata: images\output_bike03.jpg

Procesez imaginea: bike04.jpg
Obiecte detectate în images\bike04.jpg:
 - bicycle: 0.82
Am detectat 1 biciclete:
 - Bicicleta cu probabilitate: 0.82
Imagine salvata: images\output_bike04.jpg

Procesez imaginea: bike05.jpg
Obiecte detectate în images\bike05.jpg:
 - bicycle: 0.75
Am detectat 1 biciclete:
 - Bicicleta cu probabilitate: 0.75
Imagine salvata: images\output_bike05.jpg

Procesez imaginea: bike06.jpg
Obiecte detectate în images\bike06.jpg:
 - car: 0.63
 - person: 0.73
 - person: 0.80
 - cycle: 0.59
Nu s-au detectat biciclete în bike06.jpg

Pro

b,c

In [15]:

folder_path = "images"  
output_folder = "output"  


ground_truth = [
    {"image": "bike1.jpg", "boxes": [[6, 31, 409, 408]]},  
    {"image": "bike02.jpg", "boxes": [[15, 88, 381, 323]]},
    {"image": "bike03.jpg", "boxes": [[156, 145, 346, 408], [63, 145,197, 394]]},
    {"image": "bike04.jpg", "boxes": [[1, 1, 416, 416]]},
    {"image": "bike05.jpg", "boxes" : [[68, 52, 357, 345]]},
    {"image": "bike07.jpg", "boxes": [[58, 183, 298, 417]]},
    {"image": "bike08.jpg", "boxes": [[51, 1, 387, 358]]},
    {"image": "bike09.jpg", "boxes": [[4, 6, 381, 386]]},
    {"image": "bike10.jpg", "boxes": [[139, 122, 376, 407]]},
]


def detect_bicycles(image_path):
    with open(image_path, "rb") as image_stream:
        result = computervision_client.detect_objects_in_stream(image_stream)
    
    bicycles = []
    for obj in result.objects:
        if obj.object_property.lower() == "bicycle":
            rect = obj.rectangle
            
            box = [rect.x, rect.y, rect.x + rect.w, rect.y + rect.h]
            bicycles.append(box)
    
    return bicycles

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0


detected_boxes = []
files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg')) and 'bike' in f.lower()]

for file in files:
    image_path = os.path.join(folder_path, file)
    print(f"Detectez biciclete în {file}...")
    detected_bicycles = detect_bicycles(image_path)
    detected_boxes.append({"image": file, "boxes": detected_bicycles})

iou_threshold = 0.5
iou_sum = 0
iou_count = 0
total_errors = 0
total_ground_truth = sum(len(gt["boxes"]) for gt in ground_truth)  

for gt in ground_truth:
    image_name = gt["image"]
    gt_boxes = gt["boxes"]
    
    det = next((d for d in detected_boxes if d["image"] == image_name), None)
    det_boxes = det["boxes"] if det else []
    
    image_path = os.path.join(folder_path, image_name)
    image = cv2.imread(image_path)
    
    
    for box in gt_boxes:
        cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)  
        cv2.putText(image, "Ground Truth", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    for box in det_boxes:
        cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)  
        cv2.putText(image, "Azure", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    
    output_path = os.path.join(output_folder, f"output_{image_name}")
    cv2.imwrite(output_path, image)
    print(f"Imagine salvata: {output_path}")
    
    
    matched_gt = set()
    fp = 0
    
    for det_box in det_boxes:
        best_iou = 0
        best_gt_idx = -1
        for i, gt_box in enumerate(gt_boxes):
            if i in matched_gt:
                continue
            iou = calculate_iou(det_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = i
        
        if best_iou >= iou_threshold:
            matched_gt.add(best_gt_idx)
            iou_sum += best_iou
            iou_count += 1
        else:
            fp += 1
    
    fn = len(gt_boxes) - len(matched_gt)
    errors = fp + fn 
    total_errors += errors
    
    print(f"Imagine {image_name}: Numă=ar detecții={len(det_boxes)}, Numar ground truth={len(gt_boxes)}, FN={fn}, FP={fp}, IoU ={best_iou if len(det_boxes) > 0 and best_iou >= iou_threshold else 0:.2f}")

mean_iou = iou_sum / iou_count if iou_count > 0 else 0

error_rate = total_errors / total_ground_truth if total_ground_truth > 0 else 0

print(f"IoU mediu: {mean_iou:.2f}")
print(f"Error Rate: {error_rate:.2f}")

Detectez biciclete în bike02.jpg...
Detectez biciclete în bike03.jpg...
Detectez biciclete în bike04.jpg...
Detectez biciclete în bike05.jpg...
Detectez biciclete în bike06.jpg...
Detectez biciclete în bike07.jpg...
Detectez biciclete în bike08.jpg...
Detectez biciclete în bike09.jpg...
Detectez biciclete în bike1.jpg...
Detectez biciclete în bike10.jpg...
Imagine salvata: output\output_bike1.jpg
Imagine bike1.jpg: Număr detecții=1, Număr ground truth=1, FN=0, FP=0, IoU =0.94
Imagine salvata: output\output_bike02.jpg
Imagine bike02.jpg: Număr detecții=1, Număr ground truth=1, FN=0, FP=0, IoU =0.93
Imagine salvata: output\output_bike03.jpg
Imagine bike03.jpg: Număr detecții=1, Număr ground truth=2, FN=1, FP=0, IoU =0.91
Imagine salvata: output\output_bike04.jpg
Imagine bike04.jpg: Număr detecții=1, Număr ground truth=1, FN=0, FP=0, IoU =0.98
Imagine salvata: output\output_bike05.jpg
Imagine bike05.jpg: Număr detecții=1, Număr ground truth=1, FN=0, FP=0, IoU =0.89
Imagine salvata: output